In [1]:
import sys, time
import numpy as np
import cv2
import glob
from optparse import OptionParser
from pyntcloud import PyntCloud
import pandas as pd
from IPython.display import IFrame

import matplotlib.pyplot as plt

img = cv2.imread("fullImage.jpg")

if img is None:
    print("Image Not Found at: {}".format(options.image))
    exit()

hgt = img.shape[0]
wth = img.shape[1]
hwth = int(wth/2)

numPx = int(906)
# numPx = int(905)
# numPx = int(910)

center = int(hwth/2)
circlePx = int(numPx/2)

dParams = np.array([[-0.017766, 0.000774, 0.007102, -0.003659], [-0.035377, 0.037186, -0.023483, 0.004869]])
fx = np.array([301.8879, 304.6328])
fy = np.array([301.7202, 304.6480])
cx = np.array([481.2043, 479.5115])
cy = np.array([478.8208, 483.5244])

print("Done!")

Done!


In [2]:
start = time.time()
# circlePx = int(wth/2)
uPixel = np.linspace(-center, center, num=hwth, dtype=float)
vPixel = np.linspace(-center, center, num=hgt, dtype=float)

dx = np.round(cx-center)
dy = np.round(cy-center)

diff = center - circlePx

xx = uPixel/circlePx
yy = vPixel/circlePx

tmp = np.zeros((hwth,hwth), dtype = np.float32)
theta = np.array([tmp, tmp])
phi = np.array([tmp, tmp])
borders = np.array([tmp, tmp])

ii = np.array([0,0],dtype=int)
jj = np.array([1,1],dtype=int)
the = 0
# for i, x in enumerate(xx):
#     for j, y in enumerate(yy):
#         l = x**2+y**2
#         if l < 1:
#             ii = (i+dx).astype(int)
#             jj = (j+dy).astype(int)
#             the = np.sqrt(l)*np.pi/2
# #             the = the*([1,1]+dParams[:,0]*the**2+dParams[:,1]*the**4+dParams[:,2]*the**6+dParams[:,3]*the**8)
#             phi[:,ii[:],jj[:]] = np.arctan2(y,x)
#             theta[:,ii[:],jj[:]] = the
#             borders[:,ii[:],jj[:]] = 1
#     if np.mod(i,100) == 0:
#         print("Iteration {}".format(i))

borders = tmp
for i, x in enumerate(xx):
    for j, y in enumerate(yy):
        l = x**2+y**2
        if l < 1:
            the = np.sqrt(l)*np.pi/2
#             the = the*([1,1]+dParams[:,0]*the**2+dParams[:,1]*the**4+dParams[:,2]*the**6+dParams[:,3]*the**8)
            phi[:,i,j] = np.arctan2(y,x)
            theta[:,i,j] = the
            borders[i,j] = 1
    if np.mod(i,100) == 0:
        print("Iteration {}".format(i))        

# plt.imshow(theta[1])
# plt.show()
# theta = (np.pi/(2*np.max(theta)))*theta
               
print("Iteration Time: {}".format(time.time()-start))

Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Iteration Time: 3.8926539421081543


In [3]:
theta_s = np.sin(theta)
theta_c = np.cos(theta)
phi_s = np.sin(phi[0])
phi_c = np.cos(phi[0])

x = theta_s[0]*phi_c
y = theta_s[0]*phi_s
z = theta_c[0]

phi_sr = np.sin(phi[1]+np.pi)
phi_cr = np.cos(phi[1]+np.pi)

rx = -theta_s[1]*(phi_cr)
ry = theta_s[1]*(phi_sr)
rz = -theta_c[1]

count = np.count_nonzero(borders)

points = np.empty((count,3))
rPoints = np.empty((count,3))

k = 0
for i in range(hwth):
    for j in range(hwth):
        if borders[i,j] !=0:
            points[k] = [x[i,j],y[i,j],z[i,j]]
            rPoints[k] = [rx[i,j],ry[i,j],rz[i,j]]
            k += 1            
    if np.mod(i,100) == 0:
        print("Iteration {}".format(i))
        
        
print("Number of points: {}".format(points.shape[0]*2))
points = np.concatenate((points,rPoints))
# print(points.shape)
print("Done!")

Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Number of points: 1286576
Done!


In [4]:
colours = np.zeros((count,3))
rColours = np.zeros((count,3))

k = 0
for i,t in enumerate(borders!=0):
    h = ([j for j, h in enumerate(t) if h])
    for j in h:
        colours[k] = img[int(i+dx[0]),int(j+dy[0])]
        rColours[k] = img[int(i+dx[1]),int(j+hwth+dy[1])]

#         colours[k] = img[int(i+dy[0]),int(j+dx[0])]
#         rColours[k] = img[int(i+dy[1]),int(j+hwth+dx[1])]        
        k += 1
        
colours = np.concatenate((colours,rColours))
print("Done!")

Done!


In [5]:
# derp = np.column_stack((x,y,z))
cPoints = pd.DataFrame(points,columns=['x', 'y', 'z'])
cPoints[['red', 'blue', 'green']] = pd.DataFrame(colours.astype(np.uint8), index=cPoints.index)
cloud = PyntCloud(cPoints)
print("Done!")

Done!


In [8]:
# cloud.plot(point_size=0.01)